In [1]:
import numpy as np
from k_means import KMeans
from scipy.stats import multivariate_normal

class GMM:
    def __init__(self, k):
        self.k = k
        self.means = []
        self.covariances = []
        self.pis = []
        self.gammas = np.zeros((data.shape[0], self.k))

    def fit(self, data):
        """
        :params data: np.array of shape (..., dim)
                                  where dim is number of dimensions of point
        """
        self._initialize_params(data)
        old_log = self.log_likelihood(data)
        while True:
            self._E_step(data)
            self._M_step(data)
            new_log = self.log_likelihood(data)
            if (new_log - old_log) > 1:
                old_log = new_log
            else:
                break

    def _initialize_params(self, data):
        km = KMeans(self.k)
        km.fit(data)
        self.dim = data.shape[-1]
        _, self.means = km.predict(data)
        self.means = np.unique(self.means, axis = 0) #potential bug
        self.pis = np.random.rand((k,))
        self.pis = self.pis/np.sum(self.pis)
        self.covariances = np.array([np.eye(self.dim)] * self.k)

    def _E_step(data):
        for i in range(data.shape[0]):
            for k in range(self.k):
                a = multivariate_normal(self.means[k], self.covariances[k])
                self.gammas[i][k] = self.pis[k] * a.pdf(data[i])
        self.gammas = self.gammas/np.sum(self.gammas, axis = 1)[None].T

    def _M_step(data):
        for k in range(self.k):
            self.means[i] = np.dot(self.gammas[:,k].T, data)/np.sum(self.gammas[:,k])
            self.covariances[i] = np.zeros((self.dim, self.dim))
            for i in range(data.shape[0]):
                self.covariances[i] += self.gammas[i][k] * np.dot((data[i]-self.means[k])[None].T, (data[i]-self.means[k]))
            self.covariances[i] /= np.sum(self.gammas[:,k])
            self.pis[k] = np.sum(self.gammas[:,k]) / data.shape[0]

    def predict(self, data):
        new_gammas = np.zeros((data.shape[0], self.k))
        for i in range(data.shape[0]):
            for k in range(self.k):
                a = multivariate_normal(self.means[k], self.covariances[k])
                new_gammas[i][k] = self.pis[k] * a.pdf(data[i])
        new_gammas = new_gammas/np.sum(new_gammas, axis = 1)[None].T
        classes = np.argmax(new_gammas, axis = 1)
        return classes

    def get_means(self):
        return self.means.copy()

    def get_covariances(self):
        return self.covariances.copy()

    def get_pis(self):
        return self.pis.copy()
    
    def log_likelihood(self, data):
        return np.sum(np.log(np.sum(self.gammas * self.pis, axis = 1)))
                

In [5]:
np.array([np.eye(2)] * 5)

array([[[1., 0.],
        [0., 1.]],

       [[1., 0.],
        [0., 1.]],

       [[1., 0.],
        [0., 1.]],

       [[1., 0.],
        [0., 1.]],

       [[1., 0.],
        [0., 1.]]])

In [3]:
g.covariances

[]

In [15]:
a = np.load('data.npy')

In [30]:
g.fit(a)

In [31]:
c, m = g.predict(a)

In [32]:
np.unique(m, axis = 0)

array([[-7.68954908,  1.33610247],
       [-6.75782298,  4.78177533],
       [-3.1899341 ,  1.90014879],
       [ 0.63595459,  5.69909998],
       [ 4.95283136, -1.35982523]])

In [24]:
m[0]

array([ 4.95283136, -1.35982523])

In [6]:
from scipy.stats import multivariate_normal

In [13]:
a = multivariate_normal(np.array([0,0]), np.array([[1,0],[0,1]]))

In [14]:
a.pdf([0,-1])

0.09653235263005393

In [22]:
a = np.array([[1,2],[3,6]])
a = a/np.sum(a, axis = 1)[None].T

In [23]:
a

array([[0.33333333, 0.66666667],
       [0.33333333, 0.66666667]])

In [33]:
a = np.array([[1,12],[3,6]])
np.sum(a, axis = 1)

array([13,  9])

In [34]:
a

array([[ 1, 12],
       [ 3,  6]])

In [37]:
np.argmax(a, axis = 1)

array([1, 1], dtype=int64)

In [ ]:
np.random.uniform()